In this notebook, Data Generator are built for train and validation dataset, in order to feed CNN Keras models.

Such objects depends on DataPreparator_v2 object, that has been previously built.

# <font color=blus>Software engineering</font>

<img src="./img/SoftwareEngineeringDataGenerator.png">

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu(first_level=1, last_level=4, header="CNN Data generator")

# <font color=blus>0. Notebook configuration</font>

In [2]:
%matplotlib inline


%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
import pandas as pd

    Display an item from tokenized corpus

# <font color=blus>1. Loading dataset</font>

## <font color=blus> 1.1. Load dataset from manual data-preparation</font>

    This dataframe is used in ordre to complete handed data-preparation with target column.

In [24]:
df_train = pd.read_csv('./data/train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')

    List of handed data-preparation files is built.

In [4]:
import p5_util
filename = './data/dict_param_preparator.dill'
dict_param_preparator = p5_util.object_load(filename, is_verbose=True)
dict_param_preparator

p5_util.object_load : fileName= ./data/dict_param_preparator.dill


{'binary_threshold': 0.5,
 'n_sample_per_class': 71019,
 'max_row': 100,
 'nb_step': 141,
 'bulk_row': 1000,
 'remain_step': 651,
 'embedding': 300}

### <font color=blus> 1.1.1 Update target vectors into binaries labels vectors</font>

    This operation is required to build DataGenerators objects in which, target vectors 
    will be target binaries vectors.
    
    In the sequence below, DataFrame files issued from manual data-preparation are loaded.
    
    Such files contains target vectors as well as digitalized texts.
    
    Target vectors are converted into binary labeled vectors with a threshold fixed into 
    configuration parameters.
    
    Files with binary target vectors are dumped over harddisk.

In [10]:
import time
import p5_util
import p9_util
import p9_util_spacy
import DataPreparator_v2

root_filename = './data/df_dataprep_matrix_'
start_time = time.time()
bulk_id_start = 0

nb_step = dict_param_preparator['nb_step']
remain_step = dict_param_preparator['remain_step']
bulk_row = dict_param_preparator['remain_step']
binary_threshold = dict_param_preparator['bulk_row']

df_dataprep_matrix = pd.DataFrame()

list_df_data_file = list()
for bulk_id in range(bulk_id_start,nb_step) :
    item_start = bulk_id*bulk_row
    item_end = item_start+bulk_row
    filename = root_filename+str(bulk_id)+'.dill'
    list_df_data_file.append(filename)
    if True :
        df_dataprep_matrix = p5_util.object_load(filename)
        #filename = './data/df_dataprep_matrix_0_new.dill'
        
        df_dataprep_matrix['target'] = df_train['target'].loc[df_dataprep_matrix.index]

        #-----------------------------------------------------------
        # Convert target to binary label
        #-----------------------------------------------------------
        df_dataprep_matrix['target'] = p9_util.convert_vectorFloat_2_binaryLabel(df_dataprep_matrix['target'].values, \
                                                                         threshold=binary_threshold, \
                                                                         direction = 1, \
                                                                         decimal_count = 1)
        p5_util.object_dump(df_dataprep_matrix, filename, is_verbose=False)
    elapsed_time = time.time() - start_time
    #start_time = time.time()
    print("Range processed : {} --> {} Elapsed time : {:2f}".format(item_start,item_end, elapsed_time), end='\r')
    

if True :    
    if 0 < remain_step  :
        start_time = time.time()
        print("")
        bulk_id+=1
        item_start = item_end
        item_end = item_start+remain_step
        filename = root_filename+str(bulk_id)+'.dill'
        list_df_data_file.append(filename)

        if False :
            df_dataprep_matrix = p5_util.object_load(filename)
            #filename = './data/df_dataprep_matrix_0_new.dill'
            list_df_data_file.append(filename)

            df_dataprep_matrix['target'] = df_train['target'].loc[df_dataprep_matrix.index]

            #-----------------------------------------------------------
            # Convert target to binary label
            #-----------------------------------------------------------
            df_dataprep_matrix['target'] = p9_util.convert_vectorFloat_2_binaryLabel(df_dataprep_matrix['target'].values, \
                                                                             threshold=binary_threshold, \
                                                                             direction = 1, \
                                                                             decimal_count = 1)
            p5_util.object_dump(df_dataprep_matrix, filename, is_verbose=False)
        elapsed_time = time.time() - start_time
        print("Range processed : {} --> {} Elapsed time : {:2f}".format(item_start,item_end, elapsed_time), end='\r')

Range processed : 140000 --> 141000 Elapsed time : 0.011189


### <font color=blus>1.1.2. Build DataPreparator_v2 objects for train and validation</font>
    
    DataPreparator_v2 objects are required for building DataGenerators objects.
    

In [29]:
print("Number of files= {}".format(len(list_df_data_file)))
import random

set_df_data_file = set()
for idfile in range(len(list_df_data_file)//10) :
    idrand = random.randrange(0, len(list_df_data_file))
    set_df_data_file.add(list_df_data_file[idrand])

list_valid_df_data_file = list(set_df_data_file)
print("Number of file for validation= {}".format(len(list_valid_df_data_file)))

list_train_df_data_file = [train_df_data_file for train_df_data_file in list_df_data_file if train_df_data_file not in list_valid_df_data_file]
print("Number of file for validation= {}".format(len(list_train_df_data_file)))

130

#### Get configuration files issued from manual data-preparation

In [5]:
import datapreparator_config
dict_param_dataprep = datapreparator_config.dict_param_dataprep
dict_param_dataprep['max_padding_length'] = dict_param_preparator['max_row']
dict_param_dataprep['threshold'] = dict_param_preparator['binary_threshold']
dict_param_dataprep

{'other': None,
 'slice_length': 10000,
 'min_token_len': 2,
 'max_token_len': -1,
 'min_doc_len': 1,
 'max_doc_len': -1,
 'spacy_model_language': 'en_core_web_lg',
 'tokenizer': None,
 'max_padding_length': 100,
 'oov_keyword': None,
 'entity_keyword': None,
 'nb_word_most_frequent': 0,
 'is_df_copied': False,
 'is_tfidf': False,
 'threshold': 0.5}

#### Build DataPreparator_v2 objects

In [ ]:
dataPreparator_train = DataPreparator_v2.DataPreparator_v2(**dict_param_dataprep)
dataPreparator_train.list_df_data_file = list_train_df_data_file.copy()

dataPreparator_valid = DataPreparator_v2.DataPreparator_v2(**dict_param_dataprep)
dataPreparator_valid.list_df_data_file = list_valid_df_data_file.copy()


In [43]:
len(dataPreparator_train.list_df_data_file)

130

#### Target vector is loaded from files into DataPreparator_v2 attribute.

    This is a trick in order to save time for next access to target vectors.
    Target vectors are read from list of files into DataPreparator_v2 and cached into 
    object.

In [34]:
dataPreparator_valid.y[:10]

array([1., 0., 0., 1., 0., 1., 0., 1., 1., 0.])

In [35]:
dataPreparator_train.y[:10]

array([1., 1., 0., 0., 1., 0., 0., 0., 0., 1.])

#### Save DataPreparator_v2 objects for train and validation

In [37]:
import p5_util
p5_util.object_dump(dataPreparator_train, './data/dataPreparator_train_'+str(dataPreparator_train.total_row)+'.dill', is_verbose=True)
p5_util.object_dump(dataPreparator_valid, './data/dataPreparator_valid_'+str(dataPreparator_valid.total_row)+'.dill', is_verbose=True)

object_dump: file name= ./data/dataPreparator_train_129651.dill
object_dump: file name= ./data/dataPreparator_valid_12000.dill


### <font color=blus>1.1.3 Build data generator for train and validation dataset</font>

#### Load DataPreparator objects for train and validation previously built

In [7]:
import p5_util
dataPreparator_train = p5_util.object_load('./data/dataPreparator_train_129651.dill', is_verbose=True)
dataPreparator_valid = p5_util.object_load('./data/dataPreparator_valid_12000.dill', is_verbose=True)

p5_util.object_load : fileName= ./data/dataPreparator_train_129651.dill
p5_util.object_load : fileName= ./data/dataPreparator_valid_12000.dill


#### Load configuration file issued from manual data-preparation

In [8]:
import p5_util
filename = './data/dict_param_preparator.dill'
dict_param_preparator = p5_util.object_load(filename, is_verbose=True)
dict_param_preparator

p5_util.object_load : fileName= ./data/dict_param_preparator.dill


{'binary_threshold': 0.5,
 'n_sample_per_class': 71019,
 'max_row': 100,
 'nb_step': 141,
 'bulk_row': 1000,
 'remain_step': 651,
 'embedding': 300}

#### Tune configuration file for building DataGenerator object

In [13]:
import p9_util
import p9_util_config

#----------------------------------------------------------------------------
# Common parameters for DataGenerator objects.
#----------------------------------------------------------------------------
dict_param_generator = p9_util_config.dict_param_generator

nbClasses = 2
dict_param_generator['n_classes'] = nbClasses
dict_param_generator['partition_size'] = 1000
dict_param_generator['binary_threshold'] = dict_param_preparator['binary_threshold']
dict_param_generator['keras_input_dim'] = (dict_param_preparator['max_row'],dict_param_preparator['embedding'])
dict_param_generator['batch_size'] = 1000

dict_param_generator

{'data_type': 'train',
 'partition_size': 1000,
 'is_dimension_mux': False,
 'proj_dimension': None,
 'batch_size': 1000,
 'n_classes': 2,
 'n_channels': 0,
 'is_shuffle': False,
 'keras_nb_channel': 1,
 'list_keras_channel': [],
 'keras_input_dim': (100, 300),
 'binary_threshold': 0.5}

#### Build DataGenerator object used for training

    Object is dumped on harddisk once operation is completed.

In [14]:
import test_datapreparator
dict_param_generator['data_type'] = 'train'

generator = test_datapreparator.build_generator(dataPreparator_train, \
                     dict_param_generator,\
                     data_column_name='matrix_padded')


*** Partition size = 1000
p5_util.object_load : fileName= ./data/df_dataprep_matrix_0.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_1.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_2.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_3.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_4.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_5.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_6.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_7.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_8.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_9.dill
Building train partition...

p5_util.object_load : fileName= .

Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_96.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_97.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_98.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_99.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_100.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_101.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_102.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_103.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_104.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_105.dill
Building train partition...

p5_util.object_

In [15]:
generator.get_params()

{'keras_input_dim': (100, 300),
 'batch_size': 1000,
 'n_classes': 2,
 'n_channels': 0,
 'is_shuffle': False,
 'keras_nb_channel': 1,
 'list_keras_channel': [],
 'data_type': 'train',
 'proj_dimension': None,
 'is_dimension_mux': False,
 'binary_threshold': 0.5}

#### Build DataGenerator object used for validation
    Object is dumped on harddisk once operation is completed.

In [11]:
dict_param_generator['data_type'] = 'valid'
dict_param_generator

{'data_type': 'valid',
 'partition_size': 1000,
 'is_dimension_mux': False,
 'proj_dimension': None,
 'batch_size': 1000,
 'n_classes': 2,
 'n_channels': 0,
 'is_shuffle': False,
 'keras_nb_channel': 1,
 'list_keras_channel': [],
 'keras_input_dim': (100, 300),
 'binary_threshold': 0.5}

In [12]:
import test_datapreparator
generator = test_datapreparator.build_generator(dataPreparator_valid, \
                     dict_param_generator,\
                     data_column_name='matrix_padded')


*** Partition size = 1000
p5_util.object_load : fileName= ./data/df_dataprep_matrix_32.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_82.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_57.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_122.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_118.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_86.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_66.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_52.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_30.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_dataprep_matrix_79.dill
Building valid partition...

p5_util.object_load :

## <font color=blus> 1.2. Load automation dataset from `DataPreparator_v2`</font>

### <font color=blus> 1.2.1 Load configuration file issue from `DataPreparator_v2`</font>

    DataPreparator_v2 objects contain digitalized corpus for train and validation data.
    
    File names to access DataPreparator_v2 objects are built from configuration file that have 
    provided parameters for data-reparation.
    
    Keep filename from DataPreparator notebook from section 1.2.4 (Save datapreparator configuration).
    
    

In [12]:
import p5_util

filename = './data/dict_param_sequence_train_60000.dump'
dict_param_sequence = p5_util.object_load(filename)
dict_param_sequence

p5_util.object_load : fileName= ./data/dict_param_sequence_train_60000.dump


{'step': 1,
 'step_end': 2,
 'previous_step_file_name': './data/DataPreparator_v2_40Tokens_spacy__en_core_web_lg__train_60000_step1.dill',
 'dict_param_step': {1: {'dataset_filename': './data/X_y_balanced',
   'dict_param_dataprep': {'other': None,
    'slice_length': 10000,
    'min_token_len': 2,
    'max_token_len': -1,
    'min_doc_len': 1,
    'max_doc_len': -1,
    'spacy_model_language': 'en_core_web_lg',
    'tokenizer': None,
    'max_padding_length': 105,
    'oov_keyword': None,
    'entity_keyword': None,
    'nb_word_most_frequent': 0,
    'is_df_copied': False,
    'is_tfidf': False}},
  2: {'dataprep_step_filename': None,
   'dict_param_dataprep': {'other': None,
    'slice_length': 10000,
    'min_token_len': 2,
    'max_token_len': -1,
    'min_doc_len': 1,
    'max_doc_len': -1,
    'spacy_model_language': 'en_core_web_lg',
    'tokenizer': None,
    'max_padding_length': 105,
    'oov_keyword': None,
    'entity_keyword': None,
    'nb_word_most_frequent': 0,
    'is

    Build DataPreparator file name for validation dataset and read object.

In [4]:
import p5_util
import test_datapreparator

data_type = 'valid'
#n_sample = 10000
#dict_param_sequence['data_type'] = data_type
#dict_param_sequence['n_sample_valid'] = n_sample
valid_filename = test_datapreparator.datapreparator_filename(data_type, dict_param_sequence, step=2)
print(valid_filename)


dataPreparator_v2_valid = p5_util.object_load(valid_filename, is_verbose=True)
if dataPreparator_v2_valid is not None :
    # Remove duplicated indexes
    dataPreparator_v2_valid.df_data = \
    dataPreparator_v2_valid.df_data.loc[~dataPreparator_v2_valid.df_data.index.duplicated(keep='first')]
else :
    pass

./data/DataPreparator_v2_MaxTokens_spacy__en_core_web_lg__valid_10000_step2.dill
p5_util.object_load : fileName= ./data/DataPreparator_v2_MaxTokens_spacy__en_core_web_lg__valid_10000_step2.dill


    Build DataPreparator file name for train dataset and read object.

In [13]:
import p5_util
import test_datapreparator

data_type = 'train'
dict_param_sequence['data_type'] = data_type

train_filename = test_datapreparator.datapreparator_filename(data_type, dict_param_sequence, step=2)
print(train_filename)
dataPreparator_v2_train = p5_util.object_load(train_filename)
if dataPreparator_v2_train is not None :
    # Remove duplicated indexes
    dataPreparator_v2_train.df_data = \
    dataPreparator_v2_train.df_data.loc[~dataPreparator_v2_train.df_data.index.duplicated(keep='first')]
else :
    pass

./data/DataPreparator_v2_MaxTokens_spacy__en_core_web_lg__train_60000_step2.dill
p5_util.object_load : fileName= ./data/DataPreparator_v2_MaxTokens_spacy__en_core_web_lg__train_60000_step2.dill


## <font color=blus>1.2. Building targets as labels </font>

#### Update target in `DataPreparator_v2` objects and convert them as abel format

In [6]:
threshold=0.0

In [14]:
dataPreparator_v2_train.targetUpdate2BinaryLabel(threshold=threshold)

In [15]:
dataPreparator_v2_train.y[:10]

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0])

In [5]:
dataPreparator_v2_valid.targetUpdate2BinaryLabel(threshold=threshold)filename_valid_blanced

In [16]:
dataPreparator_v2_valid.y[:10]

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 0])

## <font color=blus>1.3. Building DataGenerator objects</font>

In [10]:
percent_var = 0.
print(percent_var)

#---------------------------------------------
# Used to build filename for DataPreparator_v2
# When None then the latest file name is built.
#---------------------------------------------
#step = None

0.0


In [1]:
import p9_util
import p9_util_config

#----------------------------------------------------------------------------
# Common parameters for DataGenerator objects.
#----------------------------------------------------------------------------
dict_param_generator = p9_util_config.dict_param_generator

nbClasses = 2
dict_param_generator['n_classes'] = nbClasses
dict_param_generator['partition_size'] = 1000
dict_param_generator

/home/bangui/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.
W1015 12:10:32.029115 139765984094016 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1015 12:10:32.030022 139765984094016 deprecation_wrapper.py:119] From /home/bangui/anaconda3/envs/python36/lib/python3.6/site-packages/adanet/tf_compat/__init__.py:96: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W1015 12:1

{'data_type': 'valid',
 'partition_size': 1000,
 'is_dimension_mux': False,
 'proj_dimension': None,
 'batch_size': 100,
 'n_classes': 2,
 'n_channels': 0,
 'is_shuffle': False,
 'keras_nb_channel': 1,
 'list_keras_channel': [],
 'keras_input_dim': (None, None)}

In [8]:
!rm ./data/train_X_*.*
!rm ./data/valid_X_*.*

In [17]:
#!rm ./data/train_X_*.*
data_type = 'train'
#---------------------------------------------------------------------------
# Build file name allowing to access DataPreparator_v2 object.
#  Filename issued from automated process of data preparation.
#  Filename is built from automated data-preparation configuration, 
#  implemented into dictionary dict_param_sequence.
#---------------------------------------------------------------------------
filename_datapreparator = test_datapreparator.datapreparator_filename(data_type,dict_param_sequence, step=None)

#----------------------------------------------------------------------------
# DataGenerator for train dataset
#----------------------------------------------------------------------------
dict_param_generator['partition_size'] = dict_param_sequence['dict_param_step'][2]['bulk_row']

dict_param_generator['batch_size'] = 500
dict_param_generator['data_type'] = data_type

#----------------------------------------------------------------------------
# Parameters configuration in order to build Datagenerator object for 
# train dataset.
#----------------------------------------------------------------------------
dict_cnn_preprocess={
    'filename_datapreparator' : filename_datapreparator,\
    'percent_var' : percent_var,\
}
train_generator = test_datapreparator.build_cnn_datagenerator(**dict_cnn_preprocess, \
                                                              dataPreparator_v2 = dataPreparator_v2_train)

Using DataPreparator given as parameter...

DataPreparator Dataframe shape= (59860, 1)
p5_util.object_load : fileName= ./data/df_data_df_train_60000_step2_11.dill

data_type : train
partition_size : 5000
is_dimension_mux : False
proj_dimension : None
batch_size : 500
n_classes : 2
n_channels : 0
is_shuffle : False
keras_nb_channel : 0
list_keras_channel : []
keras_input_dim : (105, 300)

Building datagenerator...

*** Partition size = 5000
p5_util.object_load : fileName= ./data/df_data_df_train_60000_step2_0.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_data_df_train_60000_step2_1.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_data_df_train_60000_step2_2.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_data_df_train_60000_step2_3.dill
Building train partition...

p5_util.object_load : fileName= ./data/df_data_df_train_60000_step2_4.dill
Building train partition...

p5_util.object_load : fileName= ./da

In [11]:
!rm ./data/valid_X_*.*    
import test_datapreparator

data_type = 'valid'
#----------------------------------------------------------------------------
# DataGenerator for validation dataset configuration
#----------------------------------------------------------------------------
dict_param_generator['partition_size'] = dict_param_sequence['dict_param_step'][2]['bulk_row']
#dict_param_generator['partition_size'] = 1000
dict_param_generator['batch_size'] = 500
dict_param_generator['data_type'] = data_type

#----------------------------------------------------------------------------
#  Filename issued from automated process of data preparation.
#  Filename is built from automated data-preparation configuration, 
# implemented into dictionary dict_param_sequence.
#----------------------------------------------------------------------------
filename_datapreparator = test_datapreparator.datapreparator_filename(data_type,dict_param_sequence, step=None)

#----------------------------------------------------------------------------
# Parameters configuration in order to build Datagenerator object for 
# validation dataset.
#----------------------------------------------------------------------------
dict_cnn_preprocess={
    'filename_datapreparator' : filename_datapreparator,\
    'percent_var' : percent_var,\
}
valid_generator = test_datapreparator.build_cnn_datagenerator(**dict_cnn_preprocess,\
                                                             dataPreparator_v2 = dataPreparator_v2_valid,\
                                                             dict_param_generator = dict_param_generator)

rm: impossible de supprimer './data/valid_X_*.*': Aucun fichier ou dossier de ce type
Using DataPreparator given as parameter...

DataPreparator Dataframe shape= (9980, 1)
p5_util.object_load : fileName= ./data/df_data_df_valid_10000_step2_1.dill

data_type : valid
partition_size : 5000
is_dimension_mux : False
proj_dimension : None
batch_size : 500
n_classes : 2
n_channels : 0
is_shuffle : False
keras_nb_channel : 0
list_keras_channel : []
keras_input_dim : (105, 300)

Building datagenerator...

*** Partition size = 5000
p5_util.object_load : fileName= ./data/df_data_df_valid_10000_step2_0.dill
Building valid partition...

p5_util.object_load : fileName= ./data/df_data_df_valid_10000_step2_1.dill
Building valid partition...



In [19]:
type(dataPreparator_v2_valid.xpca)

NoneType

#### Save / Restore DataGenerator objects

In [27]:
%matplotlib inline


%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import p5_util

filename_train = './data/train_generator.dill'
filename_valid = './data/valid_generator.dill'
if False :
    train_generator = p5_util.object_load(filename_train)
    valid_generator = p5_util.object_load(filename_valid)
else :
    p5_util.object_dump(train_generator, filename_train, is_verbose=True)
    p5_util.object_dump(valid_generator, filename_valid, is_verbose=True)
    

p5_util.object_load : fileName= ./data/train_generator.dill
p5_util.object_load : fileName= ./data/valid_generator.dill


### Switch to DataGenerator new version

In [7]:
import p5_util

filename_train = './data/train_generator.dill'
train_generator = p5_util.object_load(filename_train)

is_train = True
is_valid = not is_train

In [9]:
import p5_util

filename_valid = './data/valid_generator.dill'
valid_generator = p5_util.object_load(filename_valid)

is_train = False
is_valid = not is_train

p5_util.object_load : fileName= ./data/valid_generator.dill


In [14]:
print("(Train, Valid)= ({},{})".format(is_train, is_valid))

(Train, Valid)= (False,True)


In [12]:
#datagen_new = train_generator
if is_train :
    datagen_new = train_generator
else :     
    datagen_new = valid_generator

In [4]:
import DataGenerator
datagen = DataGenerator.DataGenerator(dict_X=datagen_new.dict_X, dict_label=datagen_new.dict_label,\
                                      nb_record = datagen_new.nb_record, other=datagen_new)

In [5]:
import DataGenerator
datagen_new = DataGenerator.DataGenerator(dict_X=datagen.dict_X, dict_label=datagen.dict_label,\
                                      nb_record = datagen.nb_record, other=datagen)

In [13]:
if is_train :
    p5_util.object_dump(datagen_new, filename_train, is_verbose=True)
else : 
    p5_util.object_dump(datagen_new, filename_valid, is_verbose=True)

object_dump: file name= ./data/valid_generator.dill
